In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
import pickle
embeddings_dataset = pickle.load(open('Distillbert_DocumentLevel_pkl', 'rb'))
embeddings_dataset

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings', 'Text'],
    num_rows: 198
})

In [ ]:
df = embeddings_dataset.to_pandas()
df

,Title,DocID,URL,Embeddings,Text
0,Child Development | Psychology Today,0,https://www.psychologytoday.com/us/basics/chil...,"[0.2803007, 0.23809318, -0.19282328, -0.159848...",Child Development | Psychology TodayUnderstand...
1,Parenting a Child with ADHD | Psychology Today,1,https://www.psychologytoday.com/us/basics/adhd...,"[-0.057591643, 0.2632406, 0.36278155, 0.245281...",Parenting a Child with ADHD | Psychology Today...
2,Raising an Adopted Child | Psychology Today,2,https://www.psychologytoday.com/us/basics/adop...,"[0.30209792, -0.043051302, -0.1418182, -0.2052...",Raising an Adopted Child | Psychology TodayRai...
3,Parenting a Child with Autism | Psychology Today,3,https://www.psychologytoday.com/us/basics/auti...,"[-0.030700075, 0.390369, 0.41788584, -0.278411...",Parenting a Child with Autism | Psychology Tod...
4,Ending Homework Battles | Psychology Today,4,https://www.psychologytoday.com/us/blog/how-ra...,"[0.26746687, 0.18432264, -0.11905984, 0.089509...",Ending Homework Battles | Psychology TodayEndi...
...,...,...,...,...,...
193,My Kid's Toddler Tantrums Are Getting Out of H...,195,https://www.parents.com/parenting/better-paren...,"[0.17019387, -0.038686477, 0.18198003, -0.1769...",My Kid's Toddler Tantrums Are Getting Out of H...
194,10 Ways to Make Feeding Your Toddler 10 Times ...,196,https://www.parents.com/recipes/scoop-on-food/...,"[0.4790843, 0.36725512, -0.06070934, -0.125382...",10 Ways to Make Feeding Your Toddler 10 Times ...
195,Separation and Social Anxiety in Kids,197,https://www.parents.com/toddlers-preschoolers/...,"[0.097834304, -0.50916183, 0.33884814, -0.6288...",Separation and Social Anxiety in KidsRaising K...
196,How Much Does My Kid Need to Eat?,198,https://www.parents.com/toddlers-preschoolers/...,"[0.9504991, 0.4124903, 0.634642, -0.18082817, ...",How Much Does My Kid Need to Eat?Family Life F...


In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
question = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]

import pandas as pd

final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])

result_dict={}


for i,question_embedding in enumerate(question):
  question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  print("samples is->")
  print(samples)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  # print("------------------------")
  # print("QUERY-> "+str(i)+" "+question[i])
  # print("------------------------")
  
  query=question[i]
  rank=1
  for _, row in samples_df.iterrows():
    title=row.Title
    docID=df.loc[df['Title'] == row.Title, 'DocID'].values[0]
    qID="q_"+str(i+1)
    dID="d_"+str(docID)
    # print(f"TITLE: {row.Title}")
    # print(f"SCORE: {row.scores}")
    # print(f"DocID: {dID}")
    # print(f"URL: {row.URL}")
    # print("=" * 50)
    # print()
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=row.scores
    else:
      result_dict[qID][dID]=row.scores
    final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

    rank+=1
#question_embedding

samples is->
{'Title': ['Anxiety Disorders  (for Parents) - Nemours KidsHealth', 'ADHD  (for Parents) - Nemours KidsHealth', 'Communication Strategies for Parents of Children with ADHD - Child Development Institute', 'Anxiety Disorders in Children: Panic Disorder', 'Separation Anxiety  (for Parents) - Nemours KidsHealth'], 'DocID': [50, 41, 69, 90, 52], 'URL': ['https://kidshealth.org/en/parents/anxiety-disorders.html#catfeelings', 'https://kidshealth.org/en/parents/adhd.html#catbehavior', 'https://childdevelopmentinfo.com/parenting/communication-strategies-parents-children-adhd/', 'https://www.webmd.com/children/guide/anxiety-disorders-in-children#091e9c5e80e5f208-1-3', 'https://kidshealth.org/en/parents/sep-anxiety.html#catfeelings'], 'Embeddings': [[-0.015090839937329292, 0.11556100845336914, 0.2705342471599579, -0.5536221861839294, -0.2987039089202881, 0.7068758010864258, 0.30997762084007263, -1.0058960914611816, -0.5905116200447083, -0.7697533965110779, 1.0948514938354492, 0.29340

In [ ]:
print(result_dict)

{'q_1': {'d_50': 323.18939208984375, 'd_41': 318.7611083984375, 'd_69': 316.2438049316406, 'd_90': 307.54669189453125, 'd_52': 301.7864685058594}, 'q_2': {'d_118': 339.51776123046875, 'd_79': 327.10113525390625, 'd_69': 338.7614440917969, 'd_76': 314.6796569824219}, 'q_3': {'d_79': 400.2335205078125, 'd_69': 386.5218200683594, 'd_131': 365.36749267578125}, 'q_4': {'d_160': 368.244140625, 'd_165': 368.244140625, 'd_164': 368.0243225097656, 'd_76': 347.4036560058594}, 'q_5': {'d_164': 354.6224060058594, 'd_161': 354.6224060058594, 'd_165': 352.9467468261719, 'd_32': 346.2237548828125}, 'q_6': {'d_121': 299.126953125, 'd_50': 299.126953125, 'd_52': 287.7478332519531, 'd_80': 250.02488708496094}, 'q_7': {'d_176': 310.0526123046875, 'd_71': 298.8423767089844, 'd_19': 297.9925842285156, 'd_124': 288.662353515625, 'd_4': 272.20245361328125}, 'q_8': {'d_79': 334.0600891113281, 'd_69': 331.7221984863281, 'd_133': 312.6273498535156, 'd_41': 312.6273498535156}, 'q_9': {'d_149': 269.3775939941406,

In [ ]:
!pip install ranx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.6/224.6 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautocorpus: filename=pyautocorpus-0.1.9-cp310-cp310-linux_x86_64.whl size=189883 sha256=4abc7eb2fea94329cb716b5aa59f4008f9dc30ccda90ed6f701cf7eb6ca1a55f
  Stored in directory: /root/.cache/pip/wheels/e2/be/43/ec1e16468c393f6e474e3be

In [ ]:
from ranx import Qrels, Run
from ranx import evaluate

In [ ]:

qrels_dict = { "q_1": { "d_65":2, "d_69":2, "d_173":2, "d_1":2, "d_41":2, 
                       "d_15": 1, "d_58": 1, "d_66": 1, "d_69": 1, "d_145": 1, "d_173" : 1},
"q_2": { "d_1":2,"d_41":2,"d_65":2,"d_15":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_3": {"d_6":2,"d_17":2,"d_80":2,"d_120":2,"d_122":2,"d_178":2,
      "d_10":1,"d_13":1,"d_14":1,"d_16":1,"d_18":1,"d_19":1,"d_25":1,"d_79":1,"d_86":1,"d_182":1},
"q_4":{"d_32":2,"d_157":2,"d_159":2,"d_162":2,"d_187":2,
      "d_188":1,"d_33":1,"d_34":1,"d_37":1,"d_52":1,"d_62":1,"d_76":1,"d_109":1,"d_163":1,"d_164":1,"d_189":1},
"q_5":{"d_23":2,"d_26":2,"d_71":2,"d_98":2,"d_183":2,"d_186":2,
      "d_22":1,"d_27":1,"d_29":1,"d_49":1,"d_60":1,"d_63":1},
"q_6":{"d_5":2,"d_8":2,"d_9":2,"d_35":2,"d_50":2,"d_52":2,"d_53":2,"d_71":2,"d_90":2,"d_125":2,"d_153":2,"d_192":2,"d_193":2,"d_194":2,
      "d_34":1,"d_45":1,"d_138":1,"d_162":1,"d_168":1},
"q_7":{"d_178":2,"d_4":2,"d_10":2,"d_14":2,"d_80":2,"d_156":2,"d_195":2,
       "d_16":1,"d_19":1,"d_25":1,"d_36":1,"d_48":1,"d_70":1,"d_86":1,"d_91":1,"d_120":1,"d_152":1,"d_167":1,"d_175":1,"d_178":1,"d_179":1
      },
"q_8":{"d_4":2,"d_6":2,"d_13":2,"d_17":2,"d_19":2,"d_178":2,
      "d_18":1,"d_36":1,"d_70":1,"d_79":1},
"q_9":{"d_22":2,"d_27":2,"d_29":2,"d_49":2,"d_60":2,"d_63":2,"d_75":2,"d_84":2,"d_99":2,"d_100":2,"d_101":2,"d_102":2,"d_103":2,"d_104":2,"d_105":2,"d_149":2,"d_150":2,
      "d_23":1,"d_26":1,"d_46":1,"d_61":1,"d_71":1,"d_88":1,"d_89":1,"d_98":1},
"q_10":{"d_91":2,"d_10":2,"d_14":2,"d_195":2,"d_36":2,"d_48":2,"d_140":2,"d_152":2,"d_154":2,"d_156":2,"d_167":2,"d_179":2,"d_195":2,
        "d_16":1,"d_18":1,"d_25":1,"d_42":1,"d_70":1,"d_95":1,"d_175":1},
"q_11":{"d_41":2,"d_65":2,"d_169":2,
      "d_5":1,"d_15":1,"d_24":1,"d_50":1,"d_54":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_12":{"d_70":2,"d_85":2,"d_174":2,"d_177":2,"d_42":2,"d_96":2,
      "d_10":1,"d_16":1,"d_18":1,"d_25":1}      
 }


run_dict = result_dict


qrels = Qrels(qrels_dict)
run = Run(run_dict)


In [ ]:

# Compute score for a single metric
# evaluate(qrels, run, "ndcg@5")
final_result=evaluate(qrels, run, ["map", "ndcg"])

# Compute scores for multiple metrics at once
# evaluate(qrels, run, ["map@5", "mrr"])
print(final_result)

{'map': 0.07738399372719547, 'ndcg': 0.14938398947673479}


In [ ]:
final_df.to_csv('DistillBert.csv')